In [2]:
import sys
sys.path.append('../')

import os
os.environ["SCRATCH"] = "/network/scratch/l/leo.gagnon"

from dataclasses import replace
import lightning as L

%load_ext autoreload
%autoreload 2

from data.diffusion import GRUDiffusionDataset, GRUDiffusionDatasetConfig
from tasks.raw_from_latent import RawFromLatent, RawFromLatentConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cfg = RawFromLatentConfig(
    pretrained_id="turo071i",
    dataset=GRUDiffusionDatasetConfig(
        context_length=(200, 200),
        pretrained_embedding=True,
        pretrained_embedding_id="turo071i",
    ),
)
task = RawFromLatent(cfg=cfg)
save_path = os.path.join(
    os.environ["SCRATCH"],
    "latent_control_log/raw_from_latent_classifier_checkpoints/model.ckpt",
)

In [4]:
trainer = L.Trainer(
    max_steps=1000,
    accelerator="gpu",
    enable_checkpointing=False,
    val_check_interval=None,
    reload_dataloaders_every_n_epochs=1,
    check_val_every_n_epoch=None,
)
trainer.fit(task)
trainer.save_checkpoint(save_path)

/home/mila/e/eric.elmoznino/latent_control/venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/e/eric.elmoznino/latent_control/venv/lib/ ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params | Mode 
------------------------------------------------
0 | out_proj | ModuleList | 83.0 K | train
------------------------------------------------
83.0 K    Trainable params
0         Non-trainable params
83.0 K    Total params
0.332     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode
/home/mila/e/eric.elmoznino/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/loop

Training: |          | 0/? [00:00<?, ?it/s]

/home/mila/e/eric.elmoznino/latent_control/notebooks/../data/hmm.py:601: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)
/home/mila/e/eric.elmoznino/latent_control/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:4252: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  return array(a, dtype=dtype, copy=bool(copy), order=order, device=device)
/home/mila/e/eric.elmoznino/latent_control/notebooks/../data/hmm.py:601: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)
/home/mila/e/eric.elmoznino/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (24) is smaller than the logging interval Trainer(log_every_n_steps=50

In [5]:
task = RawFromLatent.load_from_checkpoint(save_path, cfg=cfg)